In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.preprocessing import LabelEncoder

In [3]:


# 1. Cargar el CSV
df = pd.read_csv('tesla_coches_combinados.csv')



In [4]:
df['Precio (€)'].dtypes


dtype('O')

In [5]:


# Verificar tipo inicial de la columna
print("Tipo inicial de la columna 'Precio (€)':", df['Precio (€)'].dtypes)

# Identificar valores no numéricos
no_numericos = df[~df['Precio (€)'].astype(str).str.replace(',', '').str.replace('.', '').str.isdigit()]
if not no_numericos.empty:
    print("Valores no numéricos detectados en 'Precio (€)':")
    print(no_numericos)

# Limpiar la columna 'Precio (€)'
df['Precio (€)'] = df['Precio (€)'].str.replace(',', '').str.strip()

# Convertir a tipo numérico
df['Precio (€)'] = pd.to_numeric(df['Precio (€)'], errors='coerce')

# Eliminar filas con valores NaN después de la conversión
df = df.dropna(subset=['Precio (€)'])

# Verificar el tipo final de la columna
print("Tipo final de la columna 'Precio (€)':", df['Precio (€)'].dtypes)

# Ver los primeros valores limpios
print(df['Precio (€)'].head())


Tipo inicial de la columna 'Precio (€)': object
Tipo final de la columna 'Precio (€)': int64
0    29100
1    28600
2    28400
3    33500
4    34200
Name: Precio (€), dtype: int64


In [6]:
df['Precio (€)'].dtypes


dtype('int64')

In [7]:
df['Precio (€)'].describe()


count       623.000000
mean      58858.130016
std       24851.045888
min       25100.000000
25%       43845.000000
50%       51180.000000
75%       58972.500000
max      129025.000000
Name: Precio (€), dtype: float64

In [8]:

# Limpieza robusta de la columna 'Precio (€)'
df['Precio (€)'] = df['Precio (€)'].astype(str).str.replace(',', '').str.strip()
df['Precio (€)'] = pd.to_numeric(df['Precio (€)'], errors='coerce')
df = df.dropna(subset=['Precio (€)'])  # Eliminar filas con NaN en Precio (€)

# Convertir variables categóricas en numéricas
le_modelo = LabelEncoder()
le_color = LabelEncoder()
le_pais = LabelEncoder()

df['Modelo'] = le_modelo.fit_transform(df['Modelo'])
df['Color'] = le_color.fit_transform(df['Color'])
df['País'] = le_pais.fit_transform(df['País'])

# Separar características (X) y variable objetivo (y)
X = df.drop(columns=['Precio (€)'])
y = df['Precio (€)']

# Verificar tipos de datos en X
print("Tipos de datos en X:")
print(X.dtypes)

# Asegurar que todas las columnas de X sean numéricas
for column in X.columns:
    if X[column].dtype == 'O':  # Si es tipo objeto, convertir
        print(f"Convirtiendo la columna {column} a numérico.")
        X[column] = pd.to_numeric(X[column], errors='coerce')

# Verificar nuevamente
print("Tipos de datos en X después de la conversión:")
print(X.dtypes)

# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el modelo Random Forest
model = RandomForestRegressor(random_state=42, n_estimators=100)
model.fit(X_train, y_train)

# Evaluar el modelo
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MAE (Error Absoluto Medio): {mae}")
print(f"R2 (Coeficiente de Determinación): {r2}")

# Guardar predicciones
df_test = X_test.copy()
df_test['Precio Real (€)'] = y_test
df_test['Precio Predicho (€)'] = y_pred
df_test.to_csv('tesla_predicciones.csv', index=False)
print("Predicciones guardadas en 'tesla_predicciones.csv'.")


Tipos de datos en X:
Modelo                int64
Año                   int64
Kilometraje (kms)    object
Color                 int64
País                  int64
dtype: object
Convirtiendo la columna Kilometraje (kms) a numérico.
Tipos de datos en X después de la conversión:
Modelo                 int64
Año                    int64
Kilometraje (kms)    float64
Color                  int64
País                   int64
dtype: object
MAE (Error Absoluto Medio): 2216.8319255744264
R2 (Coeficiente de Determinación): 0.9765647635507325
Predicciones guardadas en 'tesla_predicciones.csv'.
